# Multiple Ring Neurons Experiment

## Imports

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import imageio
import datetime as dt
import os

## Equations

### Output Transform Function (Sigmoid)
$$z_{r}=f(v_{i})=\frac{1}{1+e^{-\beta (v_{i} - \mu)}}$$
$$\beta = 10; \mu = 0.2$$

In [2]:
def transform(v, beta, mu):
    return 1 / (1 + (np.e ** ((-1*beta) * (v - mu))))

### New Equations (Option 1)

Activation: $\frac{dv_i}{dt}=\frac{1}{\tau}(-\lambda u_i v_i + I_i - \gamma \sum\limits_{j \ne i}{z_j});$
$\tau = 1.0; \lambda=10$

Deactivation 
$\frac{du_i}{dt}=-\rho u_i + \frac{z_i}{c_i};$
$\rho=1.0$

Output: $z_i=f(v_i)=\frac{1}{1+e^{-\beta (v_i - \mu)}};$
$\beta = 40; \mu = 0.1$

### New Equations (Option 2)

Activation: $\frac{dv_i}{dt}=\frac{1}{\tau}(-\lambda u_i v_i + \frac{I_i}{\gamma \sum\limits_{j \ne i}{z_j}});$
$\tau = 1.0; \lambda=10$

Deactivation 
$\frac{du_i}{dt}=-\rho u_i + \frac{z_i}{c_i};$
$\rho=1.0$

Output: $z_i=f(v_i)=\frac{1}{1+e^{-\beta (v_i - \mu)}};$
$\beta = 40; \mu = 0.1$

### New Equations (Option 3)

Activation: $\frac{dv_i}{dt}=\frac{1}{\tau}(-\lambda u_i v_i + I_i);$
$\tau = 1.0; \lambda=10$

Deactivation 
$\frac{du_i}{dt}=-\rho u_i + \frac{z_i}{c_i} + \gamma \sum\limits_{j \ne i}{z_j};$
$\rho=1.0$

Output: $z_i=f(v_i)=\frac{1}{1+e^{-\beta (v_i - \mu)}};$
$\beta = 40; \mu = 0.1$

### (Old) Activity Equation
$$\frac{dv_i}{dt}=\frac{1}{\tau}(-qv_i+I_i-\gamma\sum_{j \in R}{z_j} -\alpha u_i)$$
$$\tau = 1.0; q=0.5; \gamma=???; \alpha = 0.2$$

### (Old) Deactivation Equation
$$\frac{du_i}{dt}=-au_i+\frac{z_i}{c_k}$$
$$a=0.2$$

In [3]:
# sum over all z's except for current index (in a vectorized fashion)
v = np.array([0.2,0.3,0.5])
z = np.array([0.1,0.1,0.1])
I = np.array([0.1,0.1,0.1])
print(v)
z = transform(v, BASELINE_PARAMS['beta'], BASELINE_PARAMS['mu'])
print(u)
z - np.dot(z, (1 - np.eye(3))).shape # dot product with reverse of the Identity matrix
new_v = (1 / BASELINE_PARAMS['tau']) * ((-1 * BASELINE_PARAMS['q'] * v) + I - (BASELINE_PARAMS['gamma'] * (z - np.dot(z, (1 - np.eye(3))))) - (BASELINE_PARAMS['alpha'] * u))
print(new_v)

[0.2 0.3 0.5]


NameError: name 'BASELINE_PARAMS' is not defined

In [ ]:
t = np.linspace(0, 20, 2000)

def doodle(t, state, p):
    '''
    Because we can't provide a vectorized state (i.e. state can't be 2-d in solve_ivp()),
    we hide the two vectors in state, so state is a vector of [v, u], 
    where v and u are both vectors of length `num_units`.
    
    Then, we can handle the change in v and change in u separately, 
    and concat them back together to be returned as the new state.
    '''
    # split v and u vectors
    v = state[:p['num_units']]
    print(f'v:{v.shape}')
    z = transform(v, p['beta'], p['mu'])
    print(f'z:{z.shape}')
    u = state[p['num_units']:]
    print(f'u:{u.shape}')
    # calculate dv/dt and dz/dt
    new_v = (1 / p['tau']) * ((-1 * p['q'] * v) +  p['I'] - (p['gamma'] * (z.T - np.dot(z.T, (1 - np.eye(p['num_units'])))).T) - (p['alpha'] * u)) # FIXME: u is a column vector, but should be a row vector and return a row vector (or something like that). check all of these insane .T's and stuff
    print(f'new_v:{new_v.shape}') # FIXME: why is new_v shape (2,2). related to issue above, probably!
    new_u = (-1 * p['a'] * u) + (z / p['c_k'])
    print(f'new_u:{new_u.shape}')
    # join v and u back together to be returned
    return np.concatenate((new_v, new_u))
    # return [
    #     (1 / p['tau']) * ((-1 * p['q'] * state[0]) + p['i_1'] - inhibit(transform(state[0], p['beta'], p['mu']), p['gamma']) - (p['alpha'] * state[1])),
    #     (-1 * p['a'] * state[1]) + (transform(state[0], p['beta'], p['mu']) / p['c_k'])
    # ]

## Hyperparameter Tuning

Exploring effects of $i_1$ and $c_k$

In [ ]:
import matplotlib.colors as mcolors
COLORS = mcolors.TABLEAU_COLORS
COLORS = list(COLORS.values())

In [ ]:
BASELINE_PARAMS = {
    'num_units': 2,
    'tau': 1.0,
    'q': 0.5,
    'I': np.array([0.45, 0.5]),#.reshape(2,1),
    'gamma': 0.5,
    'alpha': 0.2,
    'a': 0.2,
    'c_k': 2.0,
    'beta': 10.0,
    'mu': 0.2
}
result = solve_ivp(fun=lambda t, state: doodle(t, state, BASELINE_PARAMS), t_span=(0,20), t_eval=t, y0=np.zeros(BASELINE_PARAMS['num_units']*2), vectorized=True)
v_vec = result.y[:][:BASELINE_PARAMS['num_units']]
u_vec = result.y[:][BASELINE_PARAMS['num_units']:]
fig,ax = plt.subplots()
for i in range(BASELINE_PARAMS['num_units']):
    color = COLORS[i]
    ax.plot(t, v_vec[i], label=f'v_{i}', c=color, linestyle='dashed')
    ax.plot(t, u_vec[i], label=f'u_{i}', c=color, linestyle='dotted')
    ax.plot(t, transform(v_vec[i], BASELINE_PARAMS['beta'], BASELINE_PARAMS['mu']), label=f'z_{i}', c=color, linestyle='solid')

ax.legend()
# ax.set_ylim([0, 1])
ax.set_xlabel('t')
ax.set_title(f"Single Ring Neuron Dynamics with Baseline Parameters\ntau={BASELINE_PARAMS['tau']}|q={BASELINE_PARAMS['q']}|I={BASELINE_PARAMS['I']}|alpha={BASELINE_PARAMS['alpha']}|a={BASELINE_PARAMS['a']}|c_k={BASELINE_PARAMS['c_k']}|beta={BASELINE_PARAMS['beta']}|mu={BASELINE_PARAMS['mu']}".replace('.','p').replace('|',' - '))

v:(2, 1)
u:(2, 1)
z:(2, 1)
new_v:(2, 2)
new_z:(2, 1)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 1

In [ ]:
for i_1 in [0.0, 0.2, 0.5, 0.8, 1.0, 2.0]:
    for c_k in [0.1, 0.2, 0.5, 0.8, 1.0, 2.0]:
        params = {
            'tau': 1.0,
            'q': 0.5,
            'i_1': i_1,
            'alpha': 0.2,
            'a': 0.2,
            'c_k': c_k,
            'beta': 10.0,
            'mu': 0.2
        }
        result = solve_ivp(fun=lambda t, state: doodle(t, state, params), t_span=(0,20), t_eval=t, y0=y0)
        fig,ax = plt.subplots()
        ax.plot(t,result.y[0].T,label='v_i')
        ax.plot(t,result.y[1].T,label='u_i')
        ax.plot(t,transform(result.y[0], params['beta'], params['mu']).T,label='z_i')
        ax.legend()
        ax.set_ylim([0, 1])
        ax.set_xlabel('t')
        ax.set_title(f"tau={params['tau']}|q={params['q']}|i_1={params['i_1']}|alpha={params['alpha']}|a={params['a']}|c_k={params['c_k']}|beta={params['beta']}|mu={params['mu']}".replace('.','p').replace('|',' - '))
        plt.tight_layout()
        plt.savefig(f"plots\\tau={params['tau']}|q={params['q']}|i_1={params['i_1']}|alpha={params['alpha']}|a={params['a']}|c_k={params['c_k']}|beta={params['beta']}|mu={params['mu']}".replace('.','p').replace('|',' - '))
        plt.close()